# AI vs 인간 생성 텍스트 분류기 (LLM 기반 - 한국어 Colab 최적화)

## 🚀 Google Colab 실행 가이드 및 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치 (Colab 환경에 최적화)
!pip install -q transformers accelerate bitsandbytes peft datasets sentence-transformers
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # T4 GPU 지원 PyTorch
!pip install -q gdown # Google Drive 파일 다운로드용
!pip install -q psutil # 메모리 사용량 모니터링용

print("✅ 라이브러리 설치 완료!")

## Import 및 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import gc
import psutil
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from accelerate import Accelerator
from torch.utils.data import DataLoader
from datasets import Dataset

print("✅ 필요한 모듈 임포트 완료!")

## ⚙️ GPU 및 메모리 확인

In [ ]:
def get_memory_usage():
    process = psutil.Process(os.getpid())
    memory_mb = process.memory_info().rss / 1024 / 1024
    return memory_mb

print(f"🔍 시작 시 메모리 사용량: {get_memory_usage():.1f}MB")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
    print(f"   - 총 메모리: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
else:
    device = torch.device("cpu")
    print("⚠️ GPU 사용 불가 - CPU 모드로 전환합니다.")

gc.collect()
torch.cuda.empty_cache()
print("🧹 초기 메모리 정리 완료!")

## 📥 데이터 로드 (Google Drive 자동 다운로드)

In [ ]:
import gdown

# 파일 ID 설정 (Google Drive 공개 링크에서 추출)
# 실제 대회 데이터셋의 Google Drive ID로 교체해야 합니다.
TRAIN_FILE_ID = "1teA9GmYlIsutaDLWvCCsLeh7833t-TC_" # 예시 ID, 실제 ID로 변경 필요
TEST_FILE_ID = "1bGC_YWtNUOroHARfmzCjrL8oPcvb7Tpw"  # 예시 ID, 실제 ID로 변경 필요
SAMPLE_FILE_ID = "1ebrHVj-CtM-7aEz4OqP-bCHlazle-PKM" # 예시 ID, 실제 ID로 변경 필요

def download_from_drive(file_id, filename):
    if file_id and not os.path.exists(filename):
        url = f'https://drive.google.com/uc?id={file_id}'
        print(f"📥 {filename} 다운로드 중...")
        gdown.download(url, filename, quiet=False)
        print(f"✅ {filename} 다운로드 완료!")
    elif os.path.exists(filename):
        print(f"✅ {filename} 이미 존재함")
    else:
        print(f"⚠️ {filename} 파일 ID가 없거나 파일이 존재하지 않습니다. 수동 다운로드 필요.")

# 자동 다운로드 실행
print("📁 데이터 파일 로딩 시작...")
download_from_drive(TRAIN_FILE_ID, 'train.csv')
download_from_drive(TEST_FILE_ID, 'test.csv')
download_from_drive(SAMPLE_FILE_ID, 'sample_submission.csv')

# CSV 파일 읽기
try:
    train_df = pd.read_csv('train.csv', encoding='utf-8-sig')
    test_df = pd.read_csv('test.csv', encoding='utf-8-sig')
    sample_submission_df = pd.read_csv('sample_submission.csv', encoding='utf-8-sig')
    print("✅ 데이터셋 로드 완료!")
    print(f"   - Train shape: {train_df.shape}")
    print(f"   - Test shape: {test_df.shape}")
except Exception as e:
    print(f"❌ 데이터 로드 중 오류 발생: {e}")
    print("   - 파일이 올바르게 다운로드되었는지 확인해주세요.")
    print("   - 인코딩 문제일 수 있으니, 다른 인코딩을 시도해보세요 (예: cp949, euc-kr).")
    raise

gc.collect()
torch.cuda.empty_cache()
print("🧹 데이터 로드 후 메모리 정리 완료!")

## 📊 데이터 전처리 및 토큰화

In [ ]:
# 'paragraph_text' 컬럼을 'full_text'로 통일 (필요한 경우)
if 'paragraph_text' in test_df.columns and 'full_text' not in test_df.columns:
    test_df = test_df.rename(columns={'paragraph_text': 'full_text'})
    print("✅ Test 데이터의 'paragraph_text' 컬럼을 'full_text'로 변경했습니다.")

# 훈련 데이터와 검증 데이터 분할
X = train_df[['title', 'full_text']]
y = train_df['generated']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=42, stratify=y)

print(f"✅ 훈련 세트: {X_train.shape}, 검증 세트: {X_val.shape}")
print(f"   - 훈련 세트 AI 비율: {y_train.mean():.3f}")
print(f"   - 검증 세트 AI 비율: {y_val.mean():.3f}")

# NumPy 2.0 호환 - PyTorch Dataset 클래스 직접 구현
from torch.utils.data import Dataset as TorchDataset

class TextDataset(TorchDataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        item = {'text': str(self.texts[idx])}
        if self.labels is not None:
            item['labels'] = int(self.labels[idx])
        return item

# 텍스트 데이터 준비 (NumPy 배열 사용 안함)
train_texts = [str(title) + ' ' + str(text) for title, text in zip(X_train['title'], X_train['full_text'])]
val_texts = [str(title) + ' ' + str(text) for title, text in zip(X_val['title'], X_val['full_text'])]
test_texts = [str(title) + ' ' + str(text) for title, text in zip(test_df['title'], test_df['full_text'])]

# 라벨 준비 (NumPy 배열 사용 안함)
train_labels = y_train.tolist()
val_labels = y_val.tolist()

# PyTorch Dataset 생성
torch_train_dataset = TextDataset(train_texts, train_labels)
torch_val_dataset = TextDataset(val_texts, val_labels)
torch_test_dataset = TextDataset(test_texts)

print("✅ PyTorch Dataset 생성 완료!")

# 토크나이저 로드 (한국어 모델)
MODEL_NAME = "klue/roberta-base" # 한국어 RoBERTa 모델
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Hugging Face Dataset으로 변환 (NumPy 2.0 호환)
def convert_to_hf_dataset(torch_dataset):
    # 모든 데이터를 한 번에 변환
    data_dict = {'text': []}
    if hasattr(torch_dataset, 'labels') and torch_dataset.labels is not None:
        data_dict['labels'] = []
    
    for i in range(len(torch_dataset)):
        item = torch_dataset[i]
        data_dict['text'].append(item['text'])
        if 'labels' in item:
            data_dict['labels'].append(item['labels'])
    
    return Dataset.from_dict(data_dict)

train_dataset = convert_to_hf_dataset(torch_train_dataset)
val_dataset = convert_to_hf_dataset(torch_val_dataset)
test_dataset = convert_to_hf_dataset(torch_test_dataset)

# 토큰화 함수
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512, padding=False)

print("⏳ 데이터 토큰화 중...")
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# PyTorch 형식으로 설정
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")
tokenized_test_dataset.set_format("torch")

print("✅ 데이터 토큰화 및 형식 설정 완료!")
print(f"   - 훈련 데이터셋: {len(tokenized_train_dataset)} 샘플")
print(f"   - 검증 데이터셋: {len(tokenized_val_dataset)} 샘플")
print(f"   - 테스트 데이터셋: {len(tokenized_test_dataset)} 샘플")

gc.collect()
torch.cuda.empty_cache()
print("🧹 토큰화 후 메모리 정리 완료!")

## 🧠 모델 로드 및 PEFT (LoRA) 설정

In [ ]:
# 4비트 양자화 설정 (메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 # T4 GPU는 bfloat16 지원
)

# 모델 로드 (Sequence Classification용)
print(f"⏳ {MODEL_NAME} 모델 로드 중...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1, # 이진 분류 (0 또는 1)
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16, # 모델 dtype 설정
)

print("✅ 모델 로드 완료!")

# LoRA 설정 (PEFT)
lora_config = LoraConfig(
    r=16, # LoRA 랭크 (llm-detect-ai의 v26 설정 참고)
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    target_modules=["query", "key", "value"] # RoBERTa 모델의 어텐션 레이어
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ LoRA 설정 완료!")

gc.collect()
torch.cuda.empty_cache()
print("🧹 모델 로드 및 LoRA 설정 후 메모리 정리 완료!")

## 훈련 파라미터 및 DataLoader 설정

In [ ]:
from transformers import DataCollatorWithPadding

# 데이터 콜레이터 (동적 패딩)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# DataLoader 설정
BATCH_SIZE = 8 # T4 GPU에 적합한 배치 사이즈, 필요시 조절
GRADIENT_ACCUMULATION_STEPS = 4 # 경사 누적 단계 (실제 배치 사이즈 = BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)
NUM_EPOCHS = 3 # 훈련 에포크 수
LEARNING_RATE = 2e-5 # 학습률

train_dataloader = DataLoader(
    tokenized_train_dataset,
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

eval_dataloader = DataLoader(
    tokenized_val_dataset,
    shuffle=False,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

test_dataloader = DataLoader(
    tokenized_test_dataset,
    shuffle=False,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

print("✅ DataLoader 설정 완료!")

## 훈련 루프 (Accelerate 활용)

In [ ]:
from transformers import get_scheduler
from tqdm.auto import tqdm

accelerator = Accelerator(gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

num_training_steps = NUM_EPOCHS * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=int(num_training_steps * 0.1),
    num_training_steps=num_training_steps,
)

print("🚀 훈련 시작!")
progress_bar = tqdm(range(num_training_steps), disable=not accelerator.is_local_main_process)

model.train()
best_auc = -1.0

for epoch in range(NUM_EPOCHS):
    for step, batch in enumerate(train_dataloader):
        with accelerator.accumulate(model):
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        if accelerator.sync_gradients:
            progress_bar.update(1)
            if (step + 1) % 100 == 0: # 100 스텝마다 로깅
                accelerator.print(f"Epoch {epoch+1}, Step {step+1}/{len(train_dataloader)}, Loss: {loss.item():.4f}")

    # 에포크 종료 후 검증
    model.eval()
    all_preds = []
    all_labels = []
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        predictions = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
        labels = batch["labels"] .cpu().numpy()
        all_preds.extend(predictions)
        all_labels.extend(labels)

    auc_score = roc_auc_score(all_labels, all_preds)
    accelerator.print(f"Epoch {epoch+1} 검증 AUC: {auc_score:.4f}")

    if auc_score > best_auc:
        best_auc = auc_score
        accelerator.save_state("./best_model_checkpoint")
        accelerator.print(f"🎉 새로운 최고 AUC 달성! 모델 저장됨: {best_auc:.4f}")

    model.train()
    gc.collect()
    torch.cuda.empty_cache()

accelerator.print("✅ 훈련 완료!")

## 🔮 예측 (Inference)

In [ ]:
# 최적 모델 로드 (훈련이 완료된 후)
print("⏳ 최적 모델 로드 중...")
accelerator.load_state("./best_model_checkpoint")
model.eval()

all_test_preds = []
print("🔮 테스트 데이터 예측 중...")
for batch in tqdm(test_dataloader, disable=not accelerator.is_local_main_process):
    with torch.no_grad():
        outputs = model(**batch)
    predictions = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    all_test_preds.extend(predictions)

probs = np.array(all_test_preds)

print("✅ 예측 완료!")
print(f"   - 예측값 개수: {len(probs)}")
print(f"   - 예측값 범위: [{probs.min():.3f}, {probs.max():.3f}]")
print(f"   - 예측값 평균: {probs.mean():.3f}")

gc.collect()
torch.cuda.empty_cache()
print("🧹 예측 후 메모리 정리 완료!")

## 📤 제출 파일 생성

In [ ]:
# sample_submission 파일 읽기 (이미 로드되어 있을 수 있음)
if 'sample_submission_df' not in locals():
    sample_submission_df = pd.read_csv('sample_submission.csv', encoding='utf-8-sig')

sample_submission_df['generated'] = probs

output_filename = 'submission_llm_korean_colab.csv'
sample_submission_df.to_csv(output_filename, index=False)

print(f"✅ 제출 파일 저장 완료: {output_filename}")

# Colab에서 파일 다운로드
try:
    from google.colab import files
    files.download(output_filename)
    print(f"✅ 파일 다운로드 시작: {output_filename}")
except ImportError:
    print("로컬 환경에서 실행 중입니다. 파일이 현재 디렉터리에 저장되었습니다.")

print("🔥 모든 과정 완료!")